In [ ]:
#!pip install -U PyYAML datashader

In [ ]:
import numpy as np, pandas as pd, datashader as ds, xarray as xr
from datashader import transfer_functions as tf
from datashader.colors import inferno, viridis, Hot
from numba import jit
from datashader.utils import export_image

In [ ]:
@jit(nopython=True)
def logistica(r, x0=0.2, nvalues=100):
    #evalua = np.frompyfunc(lambda x, _: r*x*(1-x), 2, 1)
    #evalua = lambda x: r*x*(1-x), 2, 1 
    orb = np.empty(nvalues)
    orb[0] = x0
    for i in range(nvalues-1):
        x = orb[i]
        orb[i+1] = r * x * (1 - x)
    return orb

def feigenbaum(f, minimo=0, maximo=4, nr=10000, n=100000):
    ds.transfer_functions.Image.border=0
    cv = ds.Canvas(plot_width = 868, plot_height = 700)
    erres = np.linspace(minimo, maximo, nr, endpoint=True)
    xs = np.repeat(erres, n)
    ys = np.concatenate([f(r, x0=f(r)[-1], nvalues=n) for r in erres])
    df = pd.DataFrame({'x':xs, 'y':ys})
    xs, ys, erres = [None]*3  # desocupa variables temporales
    agg = cv.points(df, 'x', 'y')
    # add lines
    df1 = pd.DataFrame({'a':[2.95]*4, 'y1':[0,0.5,1, 0], 'b':[3.2]*4, 'c':[3.5]*4})
    feig = tf.shade(agg, cmap=inferno)
    return tf.set_background(feig, "black")

img = feigenbaum(logistica, 2.9, 4)
export_image(img=img, filename="feig", fmt=".png", export_path="./")

In [ ]:
@jit(nopython=True)
def lyapunov(x=0.5, r=4.0):
    for i in range(1000): x = r * x * (1 - x)
    NLYAP = 1000000
    total = 0
    for _ in range(NLYAP):
        x = r * x * (1 - x)
        total += np.log2(np.abs(r - 2 * r * x))
    return total / NLYAP
cv = ds.Canvas(plot_width = 868, plot_height = 700)
xs = np.linspace(2.9, 4, 10000)
ys = [lyapunov(r=r) for r in xs]
df = pd.DataFrame({'x':xs, 'y':ys})
#plt.plot(x, np.zeros_like(x))
agg = cv.line(df, 'x', 'y')
img = tf.shade(agg, cmap=['green'])
export_image(img=img, filename="lyap", fmt=".png", export_path="./")